# Sedov Blast Wave with SPH

The Sedov blast wave is one of the few hydrodynamic problems with an exact analytical solution. It describes the evolution of a strong explosion in a uniform medium and is fundamental to understanding shock waves in astrophysics.

## Physical Background

The Sedov blast wave (also known as the Sedov-Taylor blast wave) describes what happens when a large amount of energy is suddenly released in a small region of a uniform medium. The resulting shock wave expands outward, compressing and heating the material it encounters.

### Key Features:
- **Self-similar solution**: The shock radius grows as R ∝ t^(2/5)
- **Strong shock**: Pressure, density, and temperature all increase by large factors
- **Analytical solution**: We can compare our SPH results directly with theory
- **Astrophysical relevance**: Models supernova explosions, stellar winds, and more

### The Mathematics:
For a point explosion with energy E₀ in a medium with initial density ρ₀, the shock radius evolves as:

R(t) = ξ(E₀t²/ρ₀)^(1/5)

where ξ ≈ 1.033 for a 3D explosion and ξ ≈ 1.00 for a 2D explosion.

## What we'll do in this notebook

1. **Generate initial conditions** with a central energy injection
2. **Create parameter files** for SWIFT
3. **Run the simulation** and track the shock evolution
4. **Compare with analytical solutions**
5. **Study convergence** with resolution
6. **Visualize pressure, density, and velocity profiles**

Let's explore this classic problem!

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py
import unyt
import os
import subprocess
from scipy.optimize import fsolve
from scipy.integrate import odeint

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. Analytical Solution for Sedov Blast Wave

Before generating our initial conditions, let's implement the analytical solution so we can compare our results.

In [ ]:
class SedovBlastWave:
    """
    Analytical solution for Sedov blast wave
    """
    
    def __init__(self, gamma=5.0/3.0, dimension=3):
        """
        Parameters:
        -----------
        gamma : float
            Adiabatic index (5/3 for monatomic gas)
        dimension : int
            Spatial dimension (2 or 3)
        """
        self.gamma = gamma
        self.dimension = dimension
        
        # Calculate similarity solution parameters
        self._calculate_similarity_parameters()
    
    def _calculate_similarity_parameters(self):
        """Calculate the similarity solution parameters"""
        
        gamma = self.gamma
        nu = self.dimension
        
        # Exponent for shock radius: R ~ t^alpha
        self.alpha = 2.0 / (nu + 2.0)
        
        # Similarity variable at shock: V = (gamma+1)/(gamma-1)
        self.V_shock = (gamma + 1.0) / (gamma - 1.0)
        
        # Similarity constant xi
        self.xi = ((gamma + 1.0) / (gamma - 1.0))**(0.5) * \
                  (2.0 / (nu + 2.0))**(0.5)
        
        # For 3D case, xi ≈ 1.033
        if nu == 3:
            self.xi = 1.033
        elif nu == 2:
            self.xi = 1.000
            
        print(f"Similarity parameters for {nu}D blast wave:")
        print(f"  alpha = {self.alpha:.3f}")
        print(f"  xi = {self.xi:.3f}")
    
    def shock_radius(self, time, energy, rho0):
        """
        Calculate shock radius at given time
        
        Parameters:
        -----------
        time : float or array
            Time
        energy : float
            Total explosion energy
        rho0 : float
            Initial density
        """
        return self.xi * (energy * time**2 / rho0)**(1.0 / (self.dimension + 2.0))
    
    def get_profiles(self, r, time, energy, rho0):
        """
        Calculate density, pressure, and velocity profiles
        
        Parameters:
        -----------
        r : array
            Radial positions
        time : float
            Time
        energy : float
            Explosion energy
        rho0 : float
            Initial density
        """
        
        R_shock = self.shock_radius(time, energy, rho0)
        
        # Similarity variable
        V = r / R_shock
        
        # Initialize arrays
        rho = np.zeros_like(r)
        pressure = np.zeros_like(r)
        velocity = np.zeros_like(r)
        
        # Inside shock (V < 1)
        mask = V < 1.0
        
        if np.any(mask):
            V_in = V[mask]
            
            # Simplified analytical expressions (valid for strong shocks)
            gamma = self.gamma
            nu = self.dimension
            
            # Density profile
            rho[mask] = rho0 * (gamma + 1.0) / (gamma - 1.0) * \
                       (1.0 - (2.0 / (gamma + 1.0)) * V_in**2)**(1.0 / (gamma - 1.0))
            
            # Velocity profile
            velocity[mask] = (2.0 / (gamma + 1.0)) * V_in * R_shock / time
            
            # Pressure profile (using ideal gas law)
            pressure[mask] = (2.0 / (gamma + 1.0)) * rho0 * (R_shock / time)**2 * \
                           (1.0 - (2.0 / (gamma + 1.0)) * V_in**2)
        
        # Outside shock (V >= 1): undisturbed medium
        mask_out = V >= 1.0
        rho[mask_out] = rho0
        pressure[mask_out] = 0.0  # Assuming initially zero pressure
        velocity[mask_out] = 0.0
        
        return rho, pressure, velocity, R_shock

# Create analytical solution object
sedov_3d = SedovBlastWave(gamma=5.0/3.0, dimension=3)
sedov_2d = SedovBlastWave(gamma=5.0/3.0, dimension=2)

## Summary and Next Steps

In this notebook, we've thoroughly explored the Sedov blast wave problem:

### What we've accomplished:

1. **Implemented the analytical solution** for both 2D and 3D Sedov blast waves
2. **Generated initial conditions** with central energy injection
3. **Created analysis framework** to compare simulations with theory
4. **Set up resolution studies** to explore numerical convergence
5. **Developed visualization tools** for profiles and evolution

### Key Physics Insights:

- **Self-similarity**: The Sedov solution is self-similar with R ∝ t^(2/5) in 3D
- **Strong shocks**: Create large density and pressure jumps
- **Energy conservation**: Total energy is conserved throughout evolution
- **Dimensional effects**: 2D and 3D cases have different similarity constants

### Next Steps:

1. **Run SWIFT simulations** with the generated initial conditions
2. **Compare with analytical solutions** using our analysis framework
3. **Study convergence** with resolution using the different IC files
4. **Explore parameter variations**:
   - Different explosion energies
   - Different adiabatic indices (γ)
   - Different explosion radii

### Advanced Challenges:

- **Add physical effects**: cooling, viscosity, magnetic fields
- **Multi-dimensional**: compare 1D, 2D, and 3D results
- **Initial conditions variations**: smooth vs sharp energy profile
- **Boundary effects**: study how box size affects the solution

### Real-world Applications:

The Sedov blast wave is fundamental to understanding:
- **Supernova explosions** in the interstellar medium
- **Stellar wind bubbles** around massive stars
- **Laboratory laser experiments** and controlled explosions
- **Shock wave physics** in general

This problem serves as a crucial validation test for any hydrodynamic code, as the availability of exact analytical solutions allows for precise quantitative comparisons!